LIMPIAR AMBOS CSV

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent.parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import MOSTRADOR_SALIDA_PATH, SERVICIOS_SALIDA_PATH, ARCHIVO_MOSTRADOR_FINAL_PATH, ARCHIVO_SERVICIOS_FINAL_PATH

LIMPIO MOSTRADOR

In [2]:
import pandas as pd

# ---------------------------
# 1. Leer archivo consolidado
# ---------------------------

df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')

# ---------------------------
# 2. Eliminar ventas anuladas
# ---------------------------
if "Anulado" in df.columns:
    df = df[df["Anulado"].str.lower() == "no"]
else:
    print("No se encontró columna 'Anulado'.")

# ---------------------------
# 3. Normalización de fechas
# ---------------------------
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# ---------------------------
# 4. Agregar columnas de Año y Mes
# ---------------------------
df["anio"] = df["Fecha"].dt.year.astype("Int64")
df["mes"] = df["Fecha"].dt.month.astype("Int64")

# ---------------------------
# 5. Conversión de números
# ---------------------------
df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce').fillna(0)
df['Precio U'] = pd.to_numeric(df['Precio U'], errors='coerce').fillna(0)

# Subtotal por ítem → entero
df["subtotal_item"] = (df["Cantidad"] * df["Precio U"]).round().astype("Int64")

# ---------------------------
# 6. Validar columna idVenta
# ---------------------------
if "idVenta" not in df.columns:
    raise ValueError("No existe la columna 'idVenta' en el dataset del mostrador.")

df["idVenta"] = df["idVenta"].astype(str).str.strip()

# ---------------------------
# 7. Recalcular total real por ticket
# ---------------------------
totales = (
    df.groupby("idVenta")["subtotal_item"]
      .sum()
      .reset_index()
      .rename(columns={"subtotal_item": "TotalTicket"})
)

totales["TotalTicket"] = totales["TotalTicket"].round().astype("Int64")

# ---------------------------
# 8. Eliminar TotalVenta repetido
# ---------------------------
if "TotalVenta" in df.columns:
    df = df.drop(columns=["TotalVenta"])

# ---------------------------
# 9. Agregar total correcto por ticket
# ---------------------------
df = df.merge(totales, on="idVenta", how="left")

# ---------------------------
# 10. Seleccionar columnas útiles
# ---------------------------
columnas_utiles = [
    'Fecha', 'anio', 'mes', 'idVenta', 'Producto', 'Cantidad',
    'Cond.Pago', 'Cliente', 'CUIT', 'Impuestos',
    'Neto', 'Precio U', 'subtotal_item', 'TotalTicket'
]

df = df[columnas_utiles]

# ---------------------------
# 11. Clasificación de origen
# ---------------------------

decoracion_items = [
    'ALMOHADON', 'APOYA CUCHARA', 'AZUCARERA', 'BANCO TRES PATAS', 'BANCO RECTANGULAR',
    'COLGANTES DE CERAMICA', 'COLGANTES VARIOS', 'CORAZON CHICO', 'CORAZON GRANDE', 'CORAZON MEDIANO',
    'CUENCO CHICO', 'CUENCO MEDIANO', 'CUENCO MINI', 'CUENCO GRANDE', 'JARRA',
    'JARRITO/ MATE/ VASITO', 'JARRO', 'LECHERITA', 'MACETA CHICA', 'MACETA MEDIANA',
    'PINGUINO LITRO', 'PINGUINO MEDIO LITRO', 'PLATO 24CM', 'PLATO 30CM', 'PLATO POSTRE',
    'PORTA CUCHARA GRANDE', 'PORTA VELA', 'POSE TORTA', 'REGALERÍA', 'REGALERIA VARIOS',
    'TAZA SOLA', 'TAZA SOLA ARTESANAL', 'TAZON ARTESANAL', 'TAZON PARA LANA',
    'TELAS', 'TETERA', 'VELA', 'velas', 'TASA CON PLATO MOLDE', 'TAZA CON PLATO ARTESANAL'
]

restobar_items = ['RESTOBAR', 'restovar', 'CONFITERIA', 'Restobar']

decoracion_items = [item.lower() for item in decoracion_items]
restobar_items = [item.lower() for item in restobar_items]


def clasificar_origen(producto):
    producto_lower = str(producto).lower()

    if any(item in producto_lower for item in decoracion_items):
        return 'decoracion'
    elif any(item in producto_lower for item in restobar_items):
        return 'restobar'
    else:
        return 'mostrador'


df['origen'] = df['Producto'].apply(clasificar_origen)

# Gran bloque final: forzar números a enteros sin decimales
columnas_enteros = [
    "Cantidad", "Precio U", "subtotal_item", 
    "TotalTicket", "Neto", "Impuestos"
]

for col in columnas_enteros:
    if col in df.columns:
        df[col] = (
            pd.to_numeric(df[col], errors='coerce')
            .fillna(0)
            .round(0)
            .astype("Int64")
        )

# ---------------------------
# 12. Guardar archivo final
# ---------------------------
df.to_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, index=False, encoding='utf-8-sig')
print("Archivo limpio guardado en:", ARCHIVO_MOSTRADOR_FINAL_PATH)

# Vista previa
print(df.head())



C:\Users\juanz\AppData\Local\Temp\ipykernel_3928\1775028568.py:7: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
C:\Users\juanz\AppData\Local\Temp\ipykernel_3928\1775028568.py:20: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')


Archivo limpio guardado en: ..\..\3_data_final\ventas_mostrador_final.csv
                Fecha  anio  mes                               idVenta  \
0 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
1 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
2 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
3 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
4 2023-03-30 11:14:11  2023    3  303f00e5-91e0-4f8f-927d-fea418270f66   

                            Producto  Cantidad Cond.Pago  \
0                  ENVIO A DOMICILIO         1  .CREDITO   
1                  PROMO  LUNCH FULL        20  .CREDITO   
2  TRIPLE DE J. CRUDO , ATUN/GOURMET         2  .CREDITO   
3             TRIPLE J Y Q/ SURTIDOS         2  .CREDITO   
4           BEBIDA GASEOSA POR 2 1/4         5  .CREDITO   

                        Cliente           CUIT  Impuestos   Neto  Precio U  \
0         CASINELLI MARIO OSCAR  200777946

ELIMINO EL EFECTO INFLACIONARIO


In [4]:
import pandas as pd
from pathlib import Path

# ---------- parámetros ----------
DESTINO_ANIO = 2025
DESTINO_MES  = 11
INCLUIR_MES_ORIGEN = True
# --------------------------------

ARCHIVO_MOSTRADOR_FINAL_PATH = Path(ARCHIVO_MOSTRADOR_FINAL_PATH)

# 1) Leer ventas limpias
df = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, parse_dates=['Fecha'])

df['Anio'] = df['Fecha'].dt.year
df['Mes']  = df['Fecha'].dt.month

# 2) Inflación mensual en % (ene/2023 → nov/2025)
inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 (ene–nov)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.3, 2.0
]
infl = [x/100 for x in inflaciones_pct]

# 3) Tabla de meses
anios = [2023]*12 + [2024]*12 + [2025]*11
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,12))

tbl = pd.DataFrame({'Anio': anios, 'Mes': meses, 'inf': infl})

# Índice del destino
idx_dest = tbl.index[(tbl.Anio == DESTINO_ANIO) & (tbl.Mes == DESTINO_MES)][0]

# 4) Coeficiente compuesto (igual que antes)
coef = []
for i in range(len(tbl)):
    if i > idx_dest:
        coef.append(float('nan'))
        continue
    j_start = i if INCLUIR_MES_ORIGEN else i+1
    f = 1.0
    for j in range(j_start, idx_dest+1):
        f *= (1 + tbl.loc[j, 'inf'])
    coef.append(f)

tbl['Coef_Ajuste'] = coef

# 5) Merge
df = df.merge(tbl[['Anio','Mes','Coef_Ajuste']], on=['Anio','Mes'], how='left')

# 6) Ajustar NUEVAS columnas
df['PrecioU_Ajustado'] = (df['Precio U'] * df['Coef_Ajuste']).round(0).astype('Int64')
df['Subtotal_Ajustado'] = (df['subtotal_item'] * df['Coef_Ajuste']).round(0).astype('Int64')
df['TotalTicket_Ajustado'] = (df['TotalTicket'] * df['Coef_Ajuste']).round(0).astype('Int64')

# 7) Guardar nuevo archivo
SALIDA = ARCHIVO_MOSTRADOR_FINAL_PATH.parent / f"ventas_mostrador_final_ajustado.csv"
df.to_csv(SALIDA, index=False, encoding="utf-8-sig")

print(f"Dataset ajustado a precios de {DESTINO_MES:02d}/{DESTINO_ANIO}: {SALIDA}")
print(df.head())


C:\Users\juanz\AppData\Local\Temp\ipykernel_3928\2175745628.py:13: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ARCHIVO_MOSTRADOR_FINAL_PATH, parse_dates=['Fecha'])


Dataset ajustado a precios de 11/2025: ..\..\3_data_final\ventas_mostrador_final_ajustado.csv
                Fecha  anio  mes                               idVenta  \
0 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
1 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
2 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
3 2023-03-30 17:13:01  2023    3  d7bb8ec1-a85e-4264-83b8-bac105a2a03f   
4 2023-03-30 11:14:11  2023    3  303f00e5-91e0-4f8f-927d-fea418270f66   

                            Producto  Cantidad Cond.Pago  \
0                  ENVIO A DOMICILIO         1  .CREDITO   
1                  PROMO  LUNCH FULL        20  .CREDITO   
2  TRIPLE DE J. CRUDO , ATUN/GOURMET         2  .CREDITO   
3             TRIPLE J Y Q/ SURTIDOS         2  .CREDITO   
4           BEBIDA GASEOSA POR 2 1/4         5  .CREDITO   

                        Cliente           CUIT  Impuestos  ...  Precio U  \
0         CASINELLI MARI

LIMPIO SERVICIOS

In [2]:
import pandas as pd
from pathlib import Path

# 1) Leer ventas
df = pd.read_csv(SERVICIOS_SALIDA_PATH, parse_dates=['fecha_emision'])

# 2) Normalizar tipos (como número, sin ceros a la izquierda)
df['tipo_comprobante'] = pd.to_numeric(df['tipo_comprobante'], errors='coerce').astype('Int64')

# 3) Invertir signo SOLO para Notas de Crédito
codigos_nc = [3, 8, 13, 203]
df.loc[df['tipo_comprobante'].isin(codigos_nc), 'importe_total'] *= -1

# 4) Crear columnas Año y Mes
df['Anio'] = df['fecha_emision'].dt.year
df['Mes'] = df['fecha_emision'].dt.month

# 5) Guardar archivo final
df.to_csv(ARCHIVO_SERVICIOS_FINAL_PATH, index=False, encoding='utf-8-sig')




In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Parámetros ----------
DESTINO_ANIO  = 2025
DESTINO_MES   = 11
INCLUIR_MES_ORIGEN = True
# --------------------------------

# ============================================================
# 1) Leer ventas
# ============================================================
df = pd.read_csv(SERVICIOS_SALIDA_PATH, parse_dates=['fecha_emision'])

# ============================================================
# 2) Detectar NOTAS DE CRÉDITO y poner el importe en negativo
# ============================================================

# Códigos AFIP REALES de NC (string exacto)
TIPOS_NC = {"003", "008", "013", "113", "203", "213"}

# Mantener los tipos EXACTOS y normalizar a 3 dígitos
df['tipo_comprobante'] = df['tipo_comprobante'].astype(str).str.strip().str.zfill(3)

# Detectar NC SOLO por código correcto (no por texto)
df['es_nc'] = df['tipo_comprobante'].isin(TIPOS_NC)

# Poner en negativo las NC
df.loc[df['es_nc'], 'importe_total'] *= -1

# Año y Mes originales
df['Anio'] = df['fecha_emision'].dt.year
df['Mes']  = df['fecha_emision'].dt.month

# ============================================================
# 3) Tabla de inflación
# ============================================================

inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 ene–nov
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.3, 2.0
]

infl = [x/100 for x in inflaciones_pct]
anios = [2023]*12 + [2024]*12 + [2025]*11
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,12))

tbl = pd.DataFrame({'Anio': anios, 'Mes': meses, 'inf': infl})

# ============================================================
# 4) Cálculo del coeficiente de ajuste hasta DESTINO
# ============================================================

idx_dest = tbl.index[(tbl['Anio']==DESTINO_ANIO) & (tbl['Mes']==DESTINO_MES)][0]

coef = []
for i in range(len(tbl)):
    if i > idx_dest:
        coef.append(1)  # comprobantes posteriores al destino = no se ajustan
        continue
    j_start = i if INCLUIR_MES_ORIGEN else i + 1
    f = np.prod(1 + tbl.loc[j_start:idx_dest, 'inf'])
    coef.append(round(f, 6))

tbl['Coef_Ajuste'] = coef

# ============================================================
# 5) Merge simple: ajustar por inflación según el mes del comprobante
# ============================================================

df = df.merge(tbl[['Anio','Mes','Coef_Ajuste']], on=['Anio','Mes'], how='left')

# Ajustar importes
df['importe_ajustado'] = (df['importe_total'] * df['Coef_Ajuste']).round().astype('Int64')

# ============================================================
# 6) Guardar archivo final
# ============================================================
SALIDA = Path(ARCHIVO_SERVICIOS_FINAL_PATH).parent / "ventas_servicios_final_ajustado.csv"
df.to_csv(SALIDA, index=False, encoding='utf-8-sig')

print("✔ Archivo generado:", SALIDA)
print("   → NC detectadas solo por código AFIP")
print("   → NC en negativo correctamente")
print("   → Importes ajustados por inflación hasta", DESTINO_MES, "/", DESTINO_ANIO)



✔ Archivo generado: ..\..\3_data_final\ventas_servicios_final_ajustado.csv
   → NC detectadas solo por código AFIP
   → NC en negativo correctamente
   → Importes ajustados por inflación hasta 11 / 2025
